In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import cPickle as pickle
import multiprocessing
import scipy.special
import numpy as np
import numpy.fft as fft
import time
import os
import sys
sys.path.insert(1,'/Users/zyzdiana/GitHub/AC297r-Volume-Registration/code')

In [74]:
from conversion import rotation_matrix_fromq,rotation_matrix_from_params,rotation_matrix_to_q,angles_from_q
from utils import to_radian,res_to_rad,ax_to_w,rep_to_angle,clean
from visualize import plot_cost_func,plot_volume,plot_slices
from cost_functions import cf_ssd
from mask import sphere_mask

In [4]:
from rotation_3d import trilinear_interp,volrotate_trilinear,rotate_coords_3d,rotation_matrix_zyx
from rotation_3d import tricubic_derivatives,tricubic_interp,volrotate_tricubic
from rotation_3d import BSpline_coefficients, Bspline_interp_1d, Bspline_interp, BSpline_derivatives, volrotate_bspline
from Gauss_Newton import Gauss_Newton, Gauss_Newton1, print_results,sphere_mask,fourier_filter,get_gradient_P

In [75]:
# Get All Axes of Rotations
path = '/Users/zyzdiana/Dropbox/THESIS/July_26_2016_navs/'
ranges = ['0_5_to_2_5','3_0_to_5_0']
files = clean(os.listdir(path))
axes_dict = {}
for f in files[1:]:
    rot_ax = f.split('_')[2]
    trans_ax = f.split('_')[10]
    if not axes_dict.has_key(rot_ax):
        axes_dict[rot_ax] = trans_ax
print axes_dict
print axes_dict.keys()

{'xz': 'xy', 'y': 'z', 'yz': 'xy', 'xy': 'xy', 'x': 'z', 'z': 'z'}
['xz', 'y', 'yz', 'xy', 'x', 'z']


In [76]:
def normalize(volume, scale):
    assert((volume >= 0).all())
    return volume * scale

In [77]:
def Max_displacement(dRM, R_axis, dt):
    rad = 100
    trans_R = rad*np.sqrt(3-np.trace(dRM))
    E_max = np.sqrt(trans_R**2 + 2*trans_R*np.linalg.norm(dt-(dt.T.dot(R_axis))*R_axis) + (dt.T).dot(dt))
    return trans_R, E_max

In [78]:
def RMS_Rotation(dRM):
    return np.arccos((np.trace(dRM)-1)/2.)*180/np.pi

def RMS_Translation(dt):
    t = dt
    return np.sqrt((t.T).dot(t))
def RMS_Translation_R(dRM):
    rad = 100
    A = dRM - np.eye(3)
    return np.sqrt(0.2*rad**2*np.trace((A.T).dot(A)))

def RMS(dRM, dt, res_f):
    rad = 100
    A = dRM - np.eye(3)
    return np.sqrt(0.2*rad**2*np.trace((A.T).dot(A))+ (dt.T).dot(dt))
    
from conversion import rotation_matrix_fromq, rotation_matrix_from_params
def get_true_params(rot_angle,rot_ax):
    wy,wx,wz = ax_to_w(rot_ax)
    wy_t,wx_t,wz_t = ax_to_w(axes_dict[rot_ax])
    true_RM = rotation_matrix_fromq(rot_angle[0],wx,wy,wz)
    trans_y, trans_x, trans_z = wy_t*rot_angle[1],wx_t*rot_angle[1], wz_t*rot_angle[1]
    return [trans_x, trans_y, trans_z], true_RM

def get_params(errors, Ps, res):
    params = Ps[-1]
    RM = rotation_matrix_from_params(-params[3:])
    return  params[:3]*res, RM

In [79]:
keys = ['x']
def compute_RMS(cost_dict_6_4, cost_dict_8, cost_dict_10, grad_idx = 2, larget_trans = False):
    true_params = []
    Max_dis_6_4 = []
    Max_dis_R_6_4 = []
    RMS_ls_6_4 = []
    RMS_rotation_ls_6_4 = []
    RMS_translation_ls_6_4 = []
    RMS_trans_R_6_4 = []
    counter_6_4 = []
    
    Max_dis_8 = []
    Max_dis_R_8 = []
    RMS_ls_8 = []
    RMS_rotation_ls_8 = []
    RMS_translation_ls_8 = []
    RMS_trans_R_8 = []
    counter_8 = []
    
    Max_dis_10 = []
    Max_dis_R_10 = []
    RMS_ls_10 = []
    RMS_rotation_ls_10 = []
    RMS_translation_ls_10 = []
    RMS_trans_R_10 = []
    counter_10 = []
    for ix, rot_ax in enumerate(keys):
        for rang in ranges:
            ref = rep_to_angle(0,rang)
            for rep in xrange(1,36):
                rot_angle = rep_to_angle(rep, rang)
                if(larget_trans):
                    true_rot_angle = (rot_angle[0], rot_angle[1]*5)
                    true_params.append(true_rot_angle)
                    true_t, true_RM = get_true_params(true_rot_angle,rot_ax)
                else:
                    true_params.append(rot_angle)
                    true_t, true_RM = get_true_params(rot_angle,rot_ax)

                res = '6_4'
                res_f =  6.4
                rad = res_to_rad(res)
                errors, Ps, counter = cost_dict_6_4[grad_idx][rot_angle]
                theta = np.linalg.norm(Ps[-1][3:])*180.0/np.pi
                #errors, Ps = cost_dict_6_4[ix][grad_idx][rot_angle]
                R_axis = Ps[-1][3:]/np.linalg.norm(Ps[-1][3:])
                t , RM = get_params(errors, Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_6_4.append(E_Max)
                Max_dis_R_6_4.append(max_trans_R)
                RMS_ls_6_4.append(RMS(dR, dt, res_f))
                if(RMS(dR, dt, res_f) > E_Max):
                    print rot_ax, rang, rep, rot_angle
                    print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_6_4.append(theta)
                RMS_translation_ls_6_4.append(RMS_Translation(dt))
                RMS_trans_R_6_4.append(RMS_Translation_R(dR))
                counter_6_4.append(counter)

                res = '8'
                res_f =  8.0
                rad = res_to_rad(res)
                errors, Ps, counter = cost_dict_8[grad_idx][rot_angle]
                theta = np.linalg.norm(Ps[-1][3:])*180.0/np.pi
                #errors, Ps = cost_dict_8[ix][grad_idx][rot_angle]
                R_axis = Ps[-1][3:]/np.linalg.norm(Ps[-1][3:])
                t , RM = get_params(errors, Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_8.append(E_Max)
                Max_dis_R_8.append(max_trans_R)
                RMS_ls_8.append(RMS(dR, dt, res_f))
                if(RMS(dR, dt, res_f) > E_Max):
                    print rot_ax, rang, rep, rot_angle
                    print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_8.append(theta)
                RMS_translation_ls_8.append(RMS_Translation(dt))
                RMS_trans_R_8.append(RMS_Translation_R(dR))
                counter_8.append(counter)

                res = '10'
                res_f =  10.0
                rad = res_to_rad(res)           
                errors, Ps, counter = cost_dict_10[grad_idx][rot_angle]
                theta = np.linalg.norm(Ps[-1][3:])*180.0/np.pi
                #errors, Ps = cost_dict_10[ix][grad_idx][rot_angle]
                R_axis = Ps[-1][3:]/np.linalg.norm(Ps[-1][3:])
                t , RM = get_params(errors, Ps, res_f)
                dt = true_t + t
                dR = RM.dot(true_RM.T)
                max_trans_R, E_Max = Max_displacement(dR, R_axis, dt)
                Max_dis_10.append(E_Max)
                Max_dis_R_10.append(max_trans_R)
                RMS_ls_10.append(RMS(dR, dt, res_f))
                if(RMS(dR, dt, res_f) > E_Max):
                    print rot_ax, rang, rep, rot_angle
                    print RMS(dR, dt, res_f), E_Max
                RMS_rotation_ls_10.append(theta)
                RMS_translation_ls_10.append(RMS_Translation(dt))
                RMS_trans_R_10.append(RMS_Translation_R(dR))
                counter_10.append(counter)
                
    return [true_params,
            RMS_ls_6_4,RMS_ls_8,RMS_ls_10,
            RMS_rotation_ls_6_4,RMS_rotation_ls_8,RMS_rotation_ls_10, 
            RMS_translation_ls_6_4,RMS_translation_ls_8,RMS_translation_ls_10,
            Max_dis_6_4,Max_dis_8,Max_dis_10,
            Max_dis_R_6_4,Max_dis_R_8,Max_dis_R_10,
            RMS_trans_R_6_4, RMS_trans_R_8, RMS_trans_R_10,
            counter_6_4, counter_8, counter_10]

In [80]:
def plot_RMS(RMS_ls, interpolation):
    x = np.arange(0,70,1)
    labels = RMS_ls[0][:70]
    resolutions = ['6.4','8','10']
    for idx in xrange(3): # RMS
        plt.figure(figsize=[15,4])
        for i in xrange(6):
            plt.plot(RMS_ls[idx+1][i*70:(i+1)*70], '^' ,label = 'rotation axis: '+ axes_dict.keys()[i])
        plt.title('%s RMS %smm Resolution' % (interpolation, resolutions[idx]))
        plt.xticks(x, labels, rotation='vertical')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()

    for idx in xrange(3): # RMS for Rotations only
        plt.figure(figsize=[15,4])
        for i in xrange(6):
            plt.plot(RMS_ls[idx+4][i*70:(i+1)*70], '^' ,label = 'rotation axis: '+ axes_dict.keys()[i])
        plt.title('%s Rotation RMS %smm Resolution' % (interpolation, resolutions[idx]))
        plt.xticks(x, labels, rotation='vertical')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()
        
    for idx in xrange(3): # RMS for Translations only
        plt.figure(figsize=[15,4])
        for i in xrange(6):
            plt.plot(RMS_ls[idx+7][i*70:(i+1)*70], '^' ,label = 'translation axis: '+ axes_dict[axes_dict.keys()[i]])
        plt.title('%s Translation RMS %smm Resolution' % (interpolation, resolutions[idx]))
        plt.xticks(x, labels, rotation='vertical')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()

In [81]:
from scipy.stats import rankdata
def signed_rank_test(array1,array2):
    # Compute difference
    diff = np.array(array1)-np.array(array2)
    # record signs
    sign = np.sign(diff)
    # take absolute value
    abs_diff = abs(diff)

    # find indexes that sort an array
    indexes = np.argsort(abs_diff)

    # sorted difference and signs
    sorted_diff_all = abs_diff[indexes]
    sorted_sign_all = sign[indexes]

    # delete zero differences
    mask = sorted_diff_all >= 1e-10
    sorted_diff = sorted_diff_all[mask]
    sorted_sign = sorted_sign_all[mask]

    # Find max W
    N = len(sorted_diff)
    max_W = N*(N+1)/2.
    sigma_w = np.sqrt(N*(N+1)*(2*N+1)/6.)
    #print N, max_W

    # final difference
    final_diff = sorted_diff*sorted_sign

    # find rank
    rank = rankdata(sorted_diff)

    # final rank
    final_rank = sorted_sign*rank

    # score
    w = np.sum(final_rank)
    
    # z score
    z = (w-0.5)/sigma_w
    return N, max_W, w, z

In [165]:
date = 'july26'
model = 'AP'
ranges = ['2_5_to_12_5']

## Run Gauss Newton with BSplines on all data

In [166]:
cost_dict_bsplines_6_4 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_BSpline_GN2_6_4mm.p'%model,'rb'))
cost_dict_bsplines_8 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_BSpline_GN2_8mm.p'%model,'rb'))
cost_dict_bsplines_10 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_BSpline_GN2_10mm.p'%model,'rb'))

# RMS_bspline = compute_RMS(cost_dict_bsplines_6_4,cost_dict_bsplines_8,cost_dict_bsplines_10,1)
# RMS_bspline = compute_RMS(cost_dict_bsplines_6_4,cost_dict_bsplines_8,cost_dict_bsplines_10,2)

#plot_RMS(RMS_bspline, 'bspline')

## Run Gauss Newton with Trilinear on all data

In [167]:
cost_dict_linear_6_4 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_Linear_GN2_6_4mm.p'%model,'rb'))
cost_dict_linear_8 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_Linear_GN2_8mm.p'%model,'rb'))
cost_dict_linear_10 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_Linear_GN2_10mm.p'%model,'rb'))


# RMS_linear = compute_RMS(cost_dict_linear_6_4,cost_dict_linear_8,cost_dict_linear_10, 2)
# RMS_linear = compute_RMS(cost_dict_linear_6_4,cost_dict_linear_8,cost_dict_linear_10, 2)

#plot_RMS(RMS_linear, 'linear')

## Run Gauss Newton with Tricubic on all data

In [168]:
cost_dict_cubic_6_4 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_tricubic_GN2_6_4mm.p'%model,'rb'))
cost_dict_cubic_8 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_tricubic_GN2_8mm.p'%model,'rb'))
cost_dict_cubic_10 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/july26_dict_PE_%s_tricubic_GN2_10mm.p'%model,'rb'))

# RMS_cubic = compute_RMS(cost_dict_cubic_6_4,cost_dict_cubic_8,cost_dict_cubic_10, 2)
# RMS_cubic = compute_RMS(cost_dict_cubic_6_4,cost_dict_cubic_8,cost_dict_cubic_10, 2)

#plot_RMS(RMS_cubic, 'cubic')

In [169]:
RMS_bspline = compute_RMS(cost_dict_bsplines_6_4,cost_dict_bsplines_8,cost_dict_bsplines_10, 2)
RMS_cubic = compute_RMS(cost_dict_cubic_6_4,cost_dict_cubic_8,cost_dict_cubic_10, 2)
RMS_linear = compute_RMS(cost_dict_linear_6_4,cost_dict_linear_8,cost_dict_linear_10, 2)

In [170]:
np.max(RMS_bspline[3])

1.3101049997870975

In [158]:
RMS_bspline1 = compute_RMS(cost_dict_bsplines_6_4,cost_dict_bsplines_8,cost_dict_bsplines_10, 1)
RMS_cubic1 = compute_RMS(cost_dict_cubic_6_4,cost_dict_cubic_8,cost_dict_cubic_10, 1)
RMS_linear1 = compute_RMS(cost_dict_linear_6_4,cost_dict_linear_8,cost_dict_linear_10, 1)

In [118]:
pickle.dump(RMS_bspline,open('RMS_bspline_july26_PE_%s.p'%model,'wb'))
pickle.dump(RMS_cubic,open('RMS_cubic_july26_PE_%s.p'%model,'wb'))
pickle.dump(RMS_linear,open('RMS_linear_july26_PE_%s.p'%model,'wb'))

pickle.dump(RMS_bspline1,open('RMS_bspline1_july26_PE_%s.p'%model,'wb'))
pickle.dump(RMS_cubic1,open('RMS_cubic1_july26_PE_%s.p'%model,'wb'))
pickle.dump(RMS_linear1,open('RMS_linear1_july26_PE_%s.p'%model,'wb'))

In [152]:
PE_AP_data_ls = []
interps = ['bspline','cubic','linear']
models = ['AP','RL']
for interp in interps:
    for m in models:
        RMS_interp = pickle.load(open('RMS_%s_july26_PE_%s.p'%(interp,m),'rb'))
        print 'RMS_%s_july26_PE_%s.p'%(interp,m)
        PE_AP_data_ls.append(RMS_interp)
print len(PE_AP_data_ls)
pickle.dump(PE_AP_data_ls,open('PE_AP_data_ls.p','wb'))

PE_AP_data_ls_1 = []
interps = ['bspline1','cubic1','linear1']
dates = ['june8','july15','july26']
for interp in interps:
    for m in models:
        RMS_interp = pickle.load(open('RMS_%s_july26_PE_%s.p'%(interp,m),'rb'))
        PE_AP_data_ls_1.append(RMS_interp)
print len(PE_AP_data_ls_1)
pickle.dump(PE_AP_data_ls_1,open('PE_AP_data_ls_1.p','wb'))

RMS_bspline_july26_PE_AP.p
RMS_bspline_july26_PE_RL.p
RMS_cubic_july26_PE_AP.p
RMS_cubic_july26_PE_RL.p
RMS_linear_july26_PE_AP.p
RMS_linear_july26_PE_RL.p
6
6


In [ ]:
# compute large translation data rms
dates = ['oct13','june8','july15','july26']
for date in dates:
    cost_dict_bsplines_6_4 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_BSpline_GN2_6_4mm.p'%date,'rb'))
    cost_dict_bsplines_8 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_BSpline_GN2_8mm.p'%date,'rb'))
    cost_dict_bsplines_10 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_BSpline_GN2_10mm.p'%date,'rb'))

    cost_dict_linear_6_4 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_Linear_GN2_6_4mm.p'%date,'rb'))
    cost_dict_linear_8 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_Linear_GN2_8mm.p'%date,'rb'))
    cost_dict_linear_10 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_Linear_GN2_10mm.p'%date,'rb'))
    
    cost_dict_cubic_6_4 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_tricubic_GN2_6_4mm.p'%date,'rb'))
    cost_dict_cubic_8 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_tricubic_GN2_8mm.p'%date,'rb'))
    cost_dict_cubic_10 = pickle.load(open('/Users/zyzdiana/Dropbox/THESIS/Pickled_Results/%s_large_trans_tricubic_GN2_10mm.p'%date,'rb'))  
    
    RMS_bspline = compute_RMS(cost_dict_bsplines_6_4,cost_dict_bsplines_8,cost_dict_bsplines_10, 2)
    RMS_cubic = compute_RMS(cost_dict_cubic_6_4,cost_dict_cubic_8,cost_dict_cubic_10, 2)
    RMS_linear = compute_RMS(cost_dict_linear_6_4,cost_dict_linear_8,cost_dict_linear_10, 2)
    
    RMS_bspline1 = compute_RMS(cost_dict_bsplines_6_4,cost_dict_bsplines_8,cost_dict_bsplines_10, 1)
    RMS_cubic1 = compute_RMS(cost_dict_cubic_6_4,cost_dict_cubic_8,cost_dict_cubic_10, 1)
    RMS_linear1 = compute_RMS(cost_dict_linear_6_4,cost_dict_linear_8,cost_dict_linear_10, 1)
    
    pickle.dump(RMS_bspline,open('RMS_bspline_%s_large_trans.p'%date,'wb'))
    pickle.dump(RMS_cubic,open('RMS_cubic_%s_large_trans.p'%date,'wb'))
    pickle.dump(RMS_linear,open('RMS_linear_%s_large_trans.p'%date,'wb'))

    pickle.dump(RMS_bspline1,open('RMS_bspline1_%s_large_trans.p'%date,'wb'))
    pickle.dump(RMS_cubic1,open('RMS_cubic1_%s_large_trans.p'%date,'wb'))
    pickle.dump(RMS_linear1,open('RMS_linear1_%s_large_trans.p'%date,'wb'))

In [68]:
large_trans_data_ls = []
interps = ['bspline','cubic','linear']
dates = ['oct13','june8','july15','july26']
for interp in interps:
    for d in dates:
        RMS_interp = pickle.load(open('RMS_%s_%s_large_trans.p'%(interp,d),'rb'))
        large_trans_data_ls.append(RMS_interp)
print len(large_trans_data_ls)
pickle.dump(large_trans_data_ls,open('large_trans_data_ls.p','wb'))

large_trans_data_ls_1 = []
interps = ['bspline1','cubic1','linear1']
dates = ['oct13','june8','july15','july26']
for interp in interps:
    for d in dates:
        RMS_interp = pickle.load(open('RMS_%s_%s_large_trans.p'%(interp,d),'rb'))
        large_trans_data_ls_1.append(RMS_interp)
print len(large_trans_data_ls_1)
pickle.dump(large_trans_data_ls_1,open('large_trans_data_ls_1.p','wb'))

9
9


In [120]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
import plotly.graph_objs as go

In [62]:
c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, 9)]
colors = []
for color in c:
    colors += [color]*70
    
xx = np.ones(420)

rot_axes = ['x','y','z','xy','yz','xz']
ax_labels = []
for ax in rot_axes:
    ax_labels += [ax]*70
labels = ['10mm\nCubic\nB-spline','10mm\nTricubic','10mm\nTrilinear',
          '8mm\nCubic\nB-spline','8mm\nTricubic','8mm\nTrilinear',
          '6.4mm\nCubic\nB-spline','6.4mm\nTricubic','6.4mm\nTrilinear']

In [148]:
c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, 9)]
colors = []
for color in c:
    colors += [color]*70
    
xx = np.ones(35)

rot_axes = ['x']
ax_labels = []
for ax in rot_axes:
    ax_labels += [ax]*70
labels = ['10mm\nCubic\nB-spline','10mm\nTricubic','10mm\nTrilinear',
          '8mm\nCubic\nB-spline','8mm\nTricubic','8mm\nTrilinear',
          '6.4mm\nCubic\nB-spline','6.4mm\nTricubic','6.4mm\nTrilinear']

In [174]:
def plotly_scatter(data_ls, idx_ls, title, ylabel, num_subject = 3):
    l = []
    index = 1

    for idx in idx_ls:
        for ii, ls in enumerate(data_ls):
            subj_idx = (ii+1) % num_subject
            print subj_idx
            trace0= go.Scatter(
                x= ((xx * index) - 0.3) + (0.2*subj_idx),
                y= ls[idx],
                mode= 'markers',
                marker= dict(size= 14,
                            line= dict(width=1),
                            color= c[index-1],
                            opacity= 0.3
                           ),
                name= 'PE %s' % models[subject-1],
                text= 'x') # The hover text goes here... 
            l.append(trace0);
            if(subj_idx) == 0:
                index += 1

    layout= go.Layout(
        title= title,
        hovermode= 'closest',
        xaxis= dict(
            title= '',
            ticklen= 5,
            zeroline= False,
            gridwidth= 2,
            ticktext = labels,
            tickvals = [1,2,3,4,5,6,7,8,9]
        ),
        yaxis=dict(
            title= ylabel,
            ticklen= 5,
            gridwidth= 2,
        ),
        showlegend= False
    )
    fig= go.Figure(data=l, layout=layout)
    return fig

In [175]:
fig = plotly_scatter(PE_AP_data_ls, [3,2,1], 'RMS error PE AP RL', 'RMS', 2)
py.iplot(fig)

1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0


In [173]:
fig = plotly_scatter(PE_AP_data_ls,[12,11,10], 'Max Displacement PE RL', 'Max', 2)
py.iplot(fig)

In [74]:
fig = plotly_scatter([12,11,10], 'Maximum Displacement', 'Max')
py.iplot(fig)